# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

화형 어플리케이션, 예를 들어 챗봇을 만들 때 발생하는 문제 중 하나인 '기억'에 대해 다룹니다. 대화의 이전 부분을 기억하고 언어 모델에 그 정보를 제공하여 상호작용이 순차적으로 이루어지게 하는 것입니다. LangChain은 이러한 '기억'을 관리하는 다양한 고급 옵션을 제공합니다.

## ConversationBufferMemory

먼저 OpenAI API 키와 필요한 몇 가지 도구를 가져옵니다. 메모리 관리의 예시로 LangChain을 사용하여 챗봇 대화를 관리하는 것을 보여줍니다. LLM(Large Language Model)은 OpenAI의 챗 인터페이스로 설정하고, 메모리는 ConversationBufferMemory로 사용합니다.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [4]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [5]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [6]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

LangChain으로 대화를 시작하면 "안녕하세요, 제 이름은 Andrew입니다."라는 입력에 "안녕하세요, Andrew님. 만나서 반갑습니다."라는 답변이 나옵니다. 그런 다음 "1+1은 무엇입니까?"라는 질문에 "1+1은 2입니다."라는 답변이 나오고, 다시 "제 이름은 무엇입니까?"라고 물으면 "당신의 이름은 Andrew님입니다, 앞서 말씀드렸듯이."라고 답합니다.

In [7]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Andrew, as you mentioned earlier.


LangChain에서 실제로 어떻게 작동하는지 확인하려면 verbose 변수를 true로 변경할 수 있습니다. 그러면 LangChain이 생성하는 프롬프트를 볼 수 있습니다.

대화가 진행될수록 기억해야 할 내용이 점점 길어집니다. 따라서 memory.buffer를 출력하면 지금까지의 대화가 저장되어 있음을 확인할 수 있습니다.

In [8]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Andrew, as you mentioned earlier."}

In [9]:
memory = ConversationBufferMemory()

LangChain에서 사용하는 방식으로 대화를 저장하는 것은 ConversationBufferMemory입니다. 여기서는 명시적으로 새로운 항목들을 메모리에 추가하는 방법도 보여줍니다.

In [10]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [11]:
print(memory.buffer)

Human: Hi
AI: What's up


In [12]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [13]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [14]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

대형 언어 모델 자체는 상태가 없으며 각 API 호출도 독립적입니다. 챗봇들이 기억력을 가진 것처럼 보이는 이유는 주로 LLM에 전체 대화 컨텍스트를 제공하기 때문입니다. 따라서 메모리는 지금까지의 대화를 명시적으로 저장하고 이 정보를 LLM에 추가 컨텍스트로 제공하여 다음 대화 턴을 생성하는 데 사용합니다.

대화가 길어질수록 필요한 메모리 양이 많아지고, LLM에 많은 토큰을 보내는 비용도 증가합니다. LangChain은 이러한 대화를 저장하고 축적하는데 여러 가지 편리한 메모리 유형을 제공합니다.

## ConversationBufferWindowMemory

In [15]:
from langchain.memory import ConversationBufferWindowMemory

In [16]:
memory = ConversationBufferWindowMemory(k=1)               

In [17]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [18]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [19]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [20]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [21]:
conversation.predict(input="What is 1+1?")

'The answer to 1+1 is 2.'

In [22]:
conversation.predict(input="What is my name?")

"I'm sorry, I don't have access to that information. Could you please tell me your name?"

## ConversationTokenBufferMemory

In [23]:
#!pip install tiktoken

In [24]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [25]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [26]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

ConversationBufferMemory 외에도 다른 유형의 메모리도 있습니다. 예를 들어, 최근 몇 번의 대화만 기억하는 ConversationBufferWindowMemory 등이 있습니다.

## ConversationSummaryMemory

In [27]:
from langchain.memory import ConversationSummaryBufferMemory


In [28]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [29]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

In [30]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [31]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."

In [32]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The human asks what would be a good demo to show.\nAI: Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."}

이번 섹션에서는 LangChain이 제공하는 다양한 '기억' 유형에 대해 살펴보았습니다. 이러한 기억 유형은 대화의 이전 부분을 추적하고, 그 정보를 언어 모델에 공급하여 상호작용이 순차적으로 이루어지도록 합니다.

ConversationBufferMemory: 대화 내용을 순차적으로 저장합니다.
ConversationBufferWindowMemory: 가장 최근의 일부 대화만을 저장합니다.
ConversationSummaryBufferMemory: LLM을 사용하여 지금까지의 대화 요약을 작성하고 그것이 바로 '기억'이 됩니다.
LangChain은 각각의 기억 유형을 관리하는 방법에 따라 다르게 작동합니다. 예를 들어, ConversationBufferWindowMemory는 가장 최근의 일부 대화만을 저장하므로, 이전에 발생한 정보는 더 이상 사용할 수 없습니다. 반면, ConversationSummaryBufferMemory는 지금까지 발생한 모든 대화를 요약하여 저장하므로, 전체 대화 내용에 접근할 수 있습니다.

또한 LangChain은 벡터 데이터 메모리와 같은 추가적인 기억 유형도 지원합니다. 벡터 데이터 메모리는 단어 임베딩과 텍스트 임베딩 등과 같은 정보를 저장하며, 필요할 때 해당 정보를 검색하는 데 사용됩니다.

마지막으로 LangChain에서 여러 타입의 기억들을 함께 사용할 수 있습니다. 예를 들어 개발자들은 특정 사람들이나 엔티티들에 관한 중요한 사실들을 명시적으로 저장하기 위해 엔티티 메모리와 함께 대화 메모리를 사용할 수 있습니다.

LangChain에서 제공하는 다양한 '기억' 옵션들 덕분에 개발자들은 본질적으로 상태가 없는 LLM과 동작하는 애플리케이션(예: 챗봇)에서도 상호작용 과정에서 발생하는 주요 정보나 컨텍스트를 추적하고 보존할 수 있습니다.